# Preliminary stage

Uncertainty-aware molecular dynamics simulations of the reactants, here taking $N_2$ as an example 

Requirements:
* FLARE
* LAMMPS

In [2]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'

In [3]:
import numpy as np
import yaml
import sys
from pathlib import Path

from ase.visualize import view
from ase.io import write

sys.path.append("../../")

from mlputils.build import build_surface_FeCo, add_molecules

### Create initial configuration

$N_2$ molecule adsorbed on top of a FeCo(110) surface (or $2N$ for the products)

In [4]:
adsorbates = ['N2'] # or ['N','N']

folder = "-".join(adsorbates)+"/"
Path(folder).mkdir(exist_ok=True)

In [5]:
atoms = build_surface_FeCo(miller_index=(0,1,1),
                            layers=10, 
                            size=(4,3,1),
                            vacuum=15, 
                            fixed_layers=[0,2])

atoms = add_molecules(atoms, adsorbates, height=2., seed = 42 )

write(f'{folder}input.xyz',atoms,format='extxyz')

view(atoms,viewer='x3d')

ATOMS: 120 (48 fixed - 72 free)
Fixed atoms:  [  1   2   3   4  11  12  13  14  21  22  23  24  31  32  33  34  41  42
  43  44  51  52  53  54  61  62  63  64  71  72  73  74  81  82  83  84
  91  92  93  94 101 102 103 104 111 112 113 114]
Free atoms:  [  5   6   7   8   9  10  15  16  17  18  19  20  25  26  27  28  29  30
  35  36  37  38  39  40  45  46  47  48  49  50  55  56  57  58  59  60
  65  66  67  68  69  70  75  76  77  78  79  80  85  86  87  88  89  90
  95  96  97  98  99 100 105 106 107 108 109 110 115 116 117 118 119 120]


/leonardo/pub/userexternal/sperego0/envs/flare_v1.3.3/lib/python3.8/site-packages/ase/io/extxyz.py:311: UserWarning: Skipping unhashable information adsorbate_info
  warnings.warn('Skipping unhashable information '


### Generate FLARE input

We will generate the input starting from the configurations files stored in `configs/flare`. We will load one file per each of the four sections below. Note that there will be multiple options per each section, in the format: `section-XXX`.  

* **supercell**  : specify input configurations
* **flare_calc** : initialize the SGP or load from file (options: `init`,`file`)
* **dft_calc**   : use DFT calc (options: `qe`,`fake`)
* **otf**        : MD simulation and on-the-fly update settings (options: `lammps`,`ase`,`deal`)

In [6]:
configs_folder = '../../configs/flare/'

# load config files
settings = ['supercell',
            'flare_calc-init',      # initialize GP
            'dft_calc-qe',          # use QE as calculator
            'otf-lammps'          # use lammps for MD
            ]

config = {}
for section in settings:
    with open(f'{configs_folder}{section}.yaml') as file:
        config.update(yaml.load(file, Loader=yaml.FullLoader))

Print default settings from config files 

In [7]:
for k,v in config.items():
    print(k,":")
    print(v,"\n")

supercell :
{'file': 'input.xyz', 'format': 'extxyz', 'index': 0} 

flare_calc :
{'gp': 'SGP_Wrapper', 'kernels': [{'name': 'NormalizedDotProduct', 'sigma': 2, 'power': 2}], 'descriptors': [{'name': 'B2', 'nmax': 8, 'lmax': 3, 'cutoff_function': 'quadratic', 'radial_basis': 'chebyshev', 'cutoff_matrix': [[4, 4, 4, 4], [4, 4, 4, 4], [4, 4, 5.5, 5.5], [4, 4, 5.5, 5.5]]}], 'energy_noise': 0.1, 'forces_noise': 0.05, 'stress_noise': 0.001, 'species': [1, 7, 26, 27], 'single_atom_energies': [0, 0, 0, 0], 'cutoff': 5.5, 'variance_type': 'local', 'max_iterations': 20, 'use_mapping': True} 

dft_calc :
{'name': 'Espresso', 'kwargs': {'pseudopotentials': {'H': 'H_ONCV_PBE-1.0.oncvpsp.upf', 'N': 'N.oncvpsp.upf', 'Fe': 'Fe.pbe-spn-kjpaw_psl.0.2.1.UPF', 'Co': 'Co_pbe_v1.2.uspp.F.UPF'}, 'label': 'espresso', 'tstress': True, 'tprnfor': True, 'nosym': True, 'kpts': [2, 2, 1], 'koffset': [1, 1, 1], 'input_data': {'control': {'prefix': 'espresso', 'pseudo_dir': './', 'outdir': './tmp', 'calculation': 's

#### Initial config (supercell)

Default parameters: configs/flare/supercell.yaml

In [8]:
section = 'supercell'

config[section]['file'] = 'input.xyz'

#### Initialize Sparse GP (flare_calc)

In [9]:
species = sorted(list(set(atoms.get_atomic_numbers()))) #[1,7,26,27]  

atomic_energies_dict = { 1 : -12.662034882440556,
                         7 : -274.4786748554045,
                        26 : -4479.919873338235,
                        27 : -4061.1552386835288 }                 

cutoff = { 1 : { 1 : 4.0, 7 : 4.0, 26 : 4.0, 27 : 4.0 },
           7 : { 1 : 4.0, 7 : 4.0, 26 : 4.0, 27 : 4.0 },
          26 : { 1 : 4.0, 7 : 4.0, 26 : 5.5, 27 : 5.5 },
          27 : { 1 : 4.0, 7 : 4.0, 26 : 5.5, 27 : 5.5 }
         }

cutoff_matrix = [[cutoff[i][j] for j in species] for i in species]

In [10]:
section = 'flare_calc'

config[section]['descriptors'][0]['cutoff_matrix'] = cutoff_matrix
config[section]['cutoff'] = float(np.asarray(cutoff_matrix).max())
config[section]['single_atom_energies'] = [atomic_energies_dict[s] for s in species]

#### DFT Settings: QE (dft_calc)

In [14]:
section = 'dft_calc'

# If you want to change some option, e.g. 
#config[section]['kwargs']['input_data']['pseudo_dir'] = './'

#### OTF & MD settings: LAMMPS (otf)


In [15]:
lmp_executable = "lmp" # path to lammps executable
temp = 700
seed = 1

timestep = 0.001
nsteps   = 10000

In [16]:
section = 'otf'

config[section].update({ 'initial_velocity'  : temp,
                      'dt'                : timestep,
                      'number_of_steps'   : nsteps})
           
config[section]['md_kwargs'].update({'command' : lmp_executable,
                                 'group': [f"free id {atoms.info['free']}",
                                          f"fixed id {atoms.info['fixed']}"],
                                 'fix'  : ["1 free nve",f"2 free temp/csvr {temp} {temp} 0.1 {seed}"] })

#### Write update input

In [17]:
# write input otf to file
with open(f'{folder}input-otf.yaml', 'w') as file:
    yaml.dump(config, file, sort_keys=False)

### Perform on-the-fly training and MD

Run FLARE MD with the following command:

> flare-otf input-otf.yaml